In [6]:
import docx
import pandas as pd
import os

In [3]:
from tool.preprocessing import *

In [4]:
#From peter novig algorithm in his webpage
with open(r'./text/counts_1grams.txt') as fopen:
    f = fopen.read().split('\n')[:-1]
    
words = {}
for l in f:
    w, c = l.split('\t')
    c = int(c)
    words[w] = c + words.get(w, 0)

In [5]:
corrector = SpellCorrector(words)

In [61]:
possible_states = corrector.edit_candidates('corret')
possible_states

['croret',
 'norret',
 'corret',
 'cordet',
 'forret',
 'corre',
 'cotret',
 'correa',
 'corres',
 'corpet',
 'corrent',
 'correy',
 'corset',
 'corren',
 'corlet',
 'carret',
 'gorret',
 'correo',
 'corbet',
 'porret',
 'correr',
 'torret',
 'morret',
 'jorret',
 'couret',
 'correct',
 'cornet']

In [1]:
import bert

In [2]:
from bert import run_classifier
from bert import optimization
from bert import tokenization
from bert import modeling
import tensorflow as tf
import numpy as np

/opt/anaconda/envs/dev36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/anaconda/envs/dev36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/anaconda/envs/dev36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/anaconda/envs/dev36/lib/python3.6/site-packages/tensorflow/python/frame

/opt/anaconda/envs/dev36/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/anaconda/envs/dev36/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/anaconda/envs/dev36/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/anaconda/envs/dev36/lib/python3.6/site-packages/te

In [7]:
BERT_VOCAB = 'model/vocab.txt'
BERT_INIT_CHKPNT = 'model/bert_model.ckpt'
BERT_CONFIG = 'model/bert_config.json'

In [8]:
from bert.tokenization import WordpieceTokenizer, load_vocab, convert_by_vocab

class FullTokenizer(object):
    def __init__(self, vocab_file, do_lower_case=True):
        self.vocab = load_vocab(vocab_file)
        self.inv_vocab = {v: k for k, v in self.vocab.items()}
        self.basic_tokenizer = BasicTokenizer(do_lower_case=do_lower_case, 
                                              never_split = ['[CLS]', '[MASK]', '[SEP]'])
        self.wordpiece_tokenizer = WordpieceTokenizer(vocab=self.vocab)

    def tokenize(self, text):
        split_tokens = []
        for token in self.basic_tokenizer.tokenize(text):
            for sub_token in self.wordpiece_tokenizer.tokenize(token):
                split_tokens.append(sub_token)

        return split_tokens

    def convert_tokens_to_ids(self, tokens):
        return convert_by_vocab(self.vocab, tokens)

    def convert_ids_to_tokens(self, ids):
        return convert_by_vocab(self.inv_vocab, ids)

In [38]:
tokenizer = FullTokenizer(vocab_file=BERT_VOCAB, do_lower_case=True)

In [63]:
text = '[CLS] is the spelling corret? [SEP]'
text_mask = text.replace('corret', '**mask**')
text_mask

'[CLS] is the spelling **mask**? [SEP]'

In [64]:
def get_indices(mask, word):
    splitted = mask.split('**mask**')
    left = tokenizer.tokenize(splitted[0])
    middle = tokenizer.tokenize(word)
    right = tokenizer.tokenize(splitted[1])
    indices = [i for i in range(len(left))]
    for i in range(len(right)):
        indices.append(i + len(middle) + len(left))
    
    indices = indices[1:-1]
    tokenized = tokenizer.tokenize(mask.replace('**mask**',word))
    ids = tokenizer.convert_tokens_to_ids(tokenized)
    ids_left = tokenizer.convert_tokens_to_ids(left)
    ids_right = tokenizer.convert_tokens_to_ids(right)
    indices_word = ids_left + ids_right
    return ids, indices, indices_word[1:-1]

In [65]:
indices = [get_indices(text_mask, word) for word in possible_states]
ids, seq_ids, word_ids = list(zip(*indices))
ids

([101, 2003, 1996, 11379, 21665, 2102, 1029, 102],
 [101, 2003, 1996, 11379, 4496, 13465, 1029, 102],
 [101, 2003, 1996, 11379, 2522, 27032, 1029, 102],
 [101, 2003, 1996, 11379, 11601, 3388, 1029, 102],
 [101, 2003, 1996, 11379, 2005, 13465, 1029, 102],
 [101, 2003, 1996, 11379, 2522, 14343, 1029, 102],
 [101, 2003, 1996, 11379, 26046, 13465, 1029, 102],
 [101, 2003, 1996, 11379, 2522, 14343, 2050, 1029, 102],
 [101, 2003, 1996, 11379, 2522, 14343, 2015, 1029, 102],
 [101, 2003, 1996, 11379, 13058, 3388, 1029, 102],
 [101, 2003, 1996, 11379, 2522, 14343, 3372, 1029, 102],
 [101, 2003, 1996, 11379, 2522, 14343, 2100, 1029, 102],
 [101, 2003, 1996, 11379, 2522, 22573, 2102, 1029, 102],
 [101, 2003, 1996, 11379, 2522, 14343, 2078, 1029, 102],
 [101, 2003, 1996, 11379, 2522, 20927, 2102, 1029, 102],
 [101, 2003, 1996, 11379, 12385, 3388, 1029, 102],
 [101, 2003, 1996, 11379, 2175, 27032, 1029, 102],
 [101, 2003, 1996, 11379, 2522, 14343, 2080, 1029, 102],
 [101, 2003, 1996, 11379, 2522, 1

In [66]:
bert_config = modeling.BertConfig.from_json_file(BERT_CONFIG)

In [67]:
class Model:
    def __init__(
        self,
    ):
        self.X = tf.placeholder(tf.int32, [None, None])
        
        model = modeling.BertModel(
            config=bert_config,
            is_training=False,
            input_ids=self.X,
            use_one_hot_embeddings=False)
        
        output_layer = model.get_sequence_output()
        embedding = model.get_embedding_table()
        
        with tf.variable_scope('cls/predictions'):
            with tf.variable_scope('transform'):
                input_tensor = tf.layers.dense(
                    output_layer,
                    units = bert_config.hidden_size,
                    activation = modeling.get_activation(bert_config.hidden_act),
                    kernel_initializer = modeling.create_initializer(
                        bert_config.initializer_range
                    ),
                )
                input_tensor = modeling.layer_norm(input_tensor)
            
            output_bias = tf.get_variable(
            'output_bias',
            shape = [bert_config.vocab_size],
            initializer = tf.zeros_initializer(),
            )
            logits = tf.matmul(input_tensor, embedding, transpose_b = True)
            self.logits = tf.nn.bias_add(logits, output_bias)

In [ ]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model()

sess.run(tf.global_variables_initializer())
var_lists = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope = 'bert')

In [45]:
cls = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope = 'cls')
cls

[<tf.Variable 'cls/predictions/transform/dense/kernel:0' shape=(768, 768) dtype=float32_ref>,
 <tf.Variable 'cls/predictions/transform/dense/bias:0' shape=(768,) dtype=float32_ref>,
 <tf.Variable 'cls/predictions/transform/LayerNorm/beta:0' shape=(768,) dtype=float32_ref>,
 <tf.Variable 'cls/predictions/transform/LayerNorm/gamma:0' shape=(768,) dtype=float32_ref>,
 <tf.Variable 'cls/predictions/output_bias:0' shape=(30522,) dtype=float32_ref>]

In [46]:
saver = tf.train.Saver(var_list = var_lists + cls)
saver.restore(sess, BERT_INIT_CHKPNT)

INFO:tensorflow:Restoring parameters from model/bert_model.ckpt


In [47]:
masked_padded = tf.keras.preprocessing.sequence.pad_sequences(ids,padding='post')
masked_padded.shape

(65, 11)

In [48]:
preds = sess.run(tf.nn.softmax(model.logits), feed_dict = {model.X: masked_padded})
preds.shape

(65, 11, 30522)

In [49]:
scores = []

for no, ids in enumerate(seq_ids):
    scores.append(np.prod(preds[no, ids, word_ids[no]]))
    
scores

[0.72235274,
 0.5316523,
 0.18268654,
 0.4662075,
 0.4139198,
 0.7826236,
 0.620116,
 0.19896773,
 0.4036176,
 0.72205734,
 0.26070258,
 0.4862663,
 0.53297853,
 0.47666666,
 0.4239112,
 0.5673184,
 0.6059149,
 0.12715144,
 0.03834022,
 0.6716057,
 0.21178126,
 0.26305088,
 0.16959868,
 0.60232407,
 0.64182204,
 0.08273724,
 0.31466046,
 0.39674342,
 0.55389076,
 0.13122365,
 0.7333732,
 0.5891963,
 0.48597807,
 0.71488106,
 0.4465084,
 0.4998267,
 0.6439329,
 0.67397326,
 0.8373618,
 0.4704505,
 0.867931,
 0.051165435,
 0.08935143,
 0.5189253,
 0.4901146,
 0.71959895,
 0.61866426,
 0.82523483,
 0.38128084,
 0.2767765,
 0.25791872,
 0.47495058,
 0.8105123,
 0.43671033,
 0.41969433,
 0.9358891,
 0.48520353,
 0.80744934,
 0.5637859,
 0.15443675,
 0.38722253,
 0.29273275,
 0.30435598,
 0.5074441,
 0.55909914]

In [50]:
prob_scores = np.array(scores) / np.sum(scores)
prob_scores

array([0.0233508 , 0.01718621, 0.00590553, 0.01507064, 0.01338039,
       0.02529912, 0.02004589, 0.00643184, 0.01304736, 0.02334125,
       0.00842748, 0.01571906, 0.01722908, 0.01540874, 0.01370337,
       0.01833915, 0.01958682, 0.0041103 , 0.00123939, 0.02171035,
       0.00684605, 0.00850339, 0.00548245, 0.01947075, 0.02074756,
       0.00267457, 0.01017172, 0.01282514, 0.01790509, 0.00424194,
       0.02370705, 0.01904638, 0.01570974, 0.02310927, 0.01443384,
       0.01615741, 0.0208158 , 0.02178688, 0.02706859, 0.0152078 ,
       0.02805677, 0.00165398, 0.00288838, 0.0167748 , 0.01584346,
       0.02326178, 0.01999896, 0.02667657, 0.0123253 , 0.00894709,
       0.00833749, 0.01535327, 0.02620065, 0.01411711, 0.01356705,
       0.03025358, 0.01568471, 0.02610163, 0.01822496, 0.00499233,
       0.01251737, 0.00946289, 0.00983862, 0.01640365, 0.01807346],
      dtype=float32)

In [51]:
probs = list(zip(possible_states, prob_scores))
probs.sort(key = lambda x: x[1])  
probs[::-1]

[('cop', 0.030253582),
 ('iomp', 0.028056767),
 ('cfmp', 0.027068585),
 ('copp', 0.026676567),
 ('yomp', 0.026200647),
 ('cbmp', 0.026101634),
 ('comps', 0.025299115),
 ('compo', 0.023707047),
 ('cimp', 0.023350798),
 ('come', 0.02334125),
 ('cmop', 0.023261778),
 ('compr', 0.023109268),
 ('cmp', 0.021786882),
 ('icomp', 0.021710347),
 ('corp', 0.020815795),
 ('csmp', 0.020747561),
 ('cwmp', 0.02004589),
 ('com', 0.01999896),
 ('cnmp', 0.019586824),
 ('compy', 0.019470748),
 ('cemp', 0.01904638),
 ('como', 0.018339152),
 ('compa', 0.018224962),
 ('omp', 0.018073456),
 ('comp', 0.01790509),
 ('coma', 0.017229082),
 ('ckmp', 0.017186208),
 ('comc', 0.016774796),
 ('clmp', 0.016403655),
 ('comx', 0.016157415),
 ('jomp', 0.01584346),
 ('compl', 0.01571906),
 ('coep', 0.015709743),
 ('colp', 0.015684705),
 ('chmp', 0.015408742),
 ('fomp', 0.015353268),
 ('comb', 0.015207798),
 ('ccmp', 0.015070639),
 ('compt', 0.014433845),
 ('cocp', 0.014117112),
 ('crmp', 0.013703368),
 ('comz', 0.0135670